In [1]:
################################################################################                                                                                                                                                              
################################################################################
#
# FILE: export-couple-cbsa-combinations.ipynb
#
# BY: Dmitry Sedov 
#
# CREATED: Tue Apr 21 2020
#
# DESC: This code produces a table with CBG-restaurant pairs in a couple of 
#       CBSAs.
#
# EXEC:
#      
################################################################################
################################################################################

In [2]:
############################### Libraries ######################################

import sqlalchemy as db
import pandas as pd
import os
import numpy as np

################################################################################

In [3]:
######################### Constants and settings ##############################

# Pandas display options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

year = 2019
month = 7
days_in_month = 31
output_folder_path = '/home/user/projects/urban/data/output/spatial-demand/main_demand' 

################################################################################

In [4]:
completed_cbsa_list_file_path = os.path.join(output_folder_path, 'cbsa_list.csv')
completed_cbsa_table = pd.read_csv(completed_cbsa_list_file_path, 
                                   header = None, 
                                   names = ['cbsa'], 
                                   dtype = str)
completed_cbsa_table['completed'] = True

In [5]:
# Get restaurant and cbg count by cbsa

restaurants_by_cbsa_table_statement = """
SELECT
    cbsa,
    COUNT(sname_place_id) AS restaurant_count
FROM
    restaurants
GROUP BY
    cbsa
;
"""

cbgs_by_cbsa_table_statement = """
SELECT
    cbsa,
    COUNT(censusblockgroup) AS cbg_count
FROM
    cbgs
GROUP BY
    cbsa
;
"""

engine = db.create_engine('postgresql://{user}:{user_pass}@{host}/{dataname2}')
connection = engine.connect()

restaurants_by_cbsa_table = pd.read_sql(restaurants_by_cbsa_table_statement, 
                                        engine)

cbgs_by_cbsa_table = pd.read_sql(cbgs_by_cbsa_table_statement, 
                                 engine)

engine.dispose()

cbsa_table = pd.merge(cbgs_by_cbsa_table, 
                      restaurants_by_cbsa_table,
                      how = 'outer', 
                      on = 'cbsa', 
                      validate = 'one_to_one')

# Mark the CBSAs that are already completed
cbsa_table = pd.merge(cbsa_table, 
                      completed_cbsa_table,
                      how = 'left',
                      on = 'cbsa', 
                      validate = 'one_to_one')

cbsa_table.fillna({'completed': False}, inplace = True)
cbsa_table.sort_values('cbsa', inplace = True)

In [ ]:
selected_cbsa_list = cbsa_table.loc[(cbsa_table['completed'] == False) & 
                                    (cbsa_table['restaurant_count'] <= 5000) &
                                    (cbsa_table['restaurant_count'] >= 10) 
                                   ].reset_index(drop = True).loc[:,'cbsa'].to_list()

In [ ]:
selected_cbsa_list

In [ ]:
####### Interface to get the CBG-restaurant pairs in the selected CBSAs ########

####### SQL statements 

# Create table with selected CBSA CBGs only
create_cbgs_subset_table_statement = """
CREATE TEMPORARY TABLE subset_cbgs AS (
    SELECT 
        censusblockgroup AS cbg,
        cbsa AS cbg_cbsa,
        ST_Centroid(wkb_geometry)::geography AS cbg_centroid
    FROM 
        cbgs
    WHERE
        cbsa = '{cbsa}'
)
"""

# Create table with selected CBSA restaurants only
create_restaurants_subset_table_statement = """
CREATE TEMPORARY TABLE subset_restaurants AS (
    SELECT 
        r.sname_place_id,
        r.cbsa AS r_cbsa,
        ST_SetSRID(ST_Point(r.longitude, r.latitude), 4326)::geography AS r_location
    FROM 
        restaurants AS r
    INNER JOIN
        visits AS v
    ON
        r.sname_place_id = v.sname_place_id 
    AND
        v.year = {year} 
    AND
        v.month = {month}
    WHERE
        cbsa = '{cbsa}'
)
"""

create_devices_table_statement = """
CREATE TEMPORARY TABLE subset_devices AS (
    WITH temp_devices AS (
        SELECT
            c.cbg AS cbg,
            c.cbg_cbsa AS cbg_cbsa,
            h.number_devices_residing
        FROM
            subset_cbgs AS c
        LEFT JOIN
            home AS h
        ON
            c.cbg = h.census_block_group 
        AND
            h.year = {year} 
        AND
            h.month = {month}
    )
    SELECT 
        DISTINCT ON (cbg)
        cbg,
        cbg_cbsa,
        number_devices_residing
    FROM 
        temp_devices
    ORDER BY
        cbg,
        number_devices_residing DESC
);
"""

# Compute pairwise distances
create_distances_subset_table_statement = """
CREATE TEMPORARY TABLE subset_cbg_restaurant_dist AS (
    SELECT 
        c.cbg,
        c.cbg_cbsa,
        r.sname_place_id,
        r.r_cbsa,
        ST_Distance(c.cbg_centroid, r.r_location) as distance
    FROM
        subset_cbgs AS c,
        subset_restaurants AS r
);
ALTER TABLE subset_cbg_restaurant_dist 
ADD COLUMN number_devices_residing INTEGER;

UPDATE subset_cbg_restaurant_dist p
SET number_devices_residing = d.number_devices_residing
FROM subset_devices d
WHERE p.cbg = d.cbg;
"""

export_distances_subset_table = """
COPY subset_cbg_restaurant_dist TO STDOUT WITH (FORMAT CSV, HEADER);
"""

distances_subset_table_statement = """
SELECT 
    *
FROM
    subset_cbg_restaurant_dist
;
"""

devices_table_statement = """
SELECT 
    *
FROM 
    subset_devices
;
"""

# Create a table with CBGs' device count and CBSA affiliation
restaurants_table_statement = """
SELECT
    r.sname_place_id AS sname_place_id,
    r.zip_code,
    r.cbg AS r_cbg,
    r.cbsa AS r_cbsa,
    r.price,
    r.rating,
    r.naics_code,
    r.area_m2,
    r.brands,
    r.categories,
    r.includes_parking_lot,
    r.total_minutes_open,
    r.parent_sname_place_id IS NOT NULL AS is_part,
    v.raw_visit_counts AS raw_visit_counts
FROM 
    restaurants AS r
INNER JOIN
    visits AS v
ON
    r.sname_place_id = v.sname_place_id 
AND
    v.year = {year} 
AND
    v.month = {month}
WHERE
    r.cbsa = '{cbsa}'
;
"""

# Create table with selected CBSA establishments only
create_establishments_subset_table_statement = """
CREATE TEMPORARY TABLE subset_establishments AS (
    SELECT 
        e.sname_place_id,
        e.cbg
    FROM 
        establishments AS e
    INNER JOIN
        visits AS v
    ON
        e.sname_place_id = v.sname_place_id 
    AND
        v.year = {year} 
    AND
        v.month = {month}
    WHERE
        cbsa = '{cbsa}'
)
"""

establishments_subset_by_cbg_table_statement = """
SELECT 
    cbg,
    COUNT(sname_place_id) AS est_nearby
FROM 
    subset_establishments
GROUP BY
    cbg
;
"""

################################################################################


####### Function to work on a single CBSA
def create_combinations(selected_cbsa):
    
    print(f'Working with {selected_cbsa}.')
    
    cbsa_folder_path = os.path.join(output_folder_path, f'cbsa{selected_cbsa}')
    if not os.path.exists(cbsa_folder_path):
        os.makedirs(cbsa_folder_path)
        
    cbsa_pairs_file_path = os.path.join(cbsa_folder_path, f'pairs{selected_cbsa}.csv')
    
    engine = db.create_engine('postgresql://{user}:{user_pass}@{host}/{dataname2}')
    
    # Format SQL statements
    l_create_cbgs_subset_table_statement = create_cbgs_subset_table_statement.format(cbsa = selected_cbsa)
    l_create_restaurants_subset_table_statement = create_restaurants_subset_table_statement.format(cbsa = selected_cbsa,
                                                                                                   year = year,
                                                                                                   month = month)
    l_create_devices_table_statement = create_devices_table_statement.format(year = year, month = month)
    l_restaurants_table_statement = restaurants_table_statement.format(cbsa = selected_cbsa, 
                                                                       year = year, 
                                                                       month = month)
    l_create_establishments_subset_table_statement = (
        create_establishments_subset_table_statement.format(cbsa = selected_cbsa,
                                                            year = year,
                                                            month = month)
    )
    l_export_distances_subset_table = export_distances_subset_table
    
    print(f'Getting cbgs, restaurants and establishments.')
    # Build a table of cbgs in selected cbsa
    result = engine.execute(l_create_cbgs_subset_table_statement)

    # Build a table with restaurants in selected cbsa
    result = engine.execute(l_create_restaurants_subset_table_statement)
    
    # Build a table with establishments in selected cbsa
    result = engine.execute(l_create_establishments_subset_table_statement)

    print(f'Computing distances.')
    result = engine.execute(l_create_devices_table_statement)
    # Compute distances
    result = engine.execute(create_distances_subset_table_statement)
    print(f'Exporting pairs.')
    with open(cbsa_pairs_file_path, 'w') as pairs_file:
        connection = engine.raw_connection()
        cursor = connection.cursor()
        cursor.copy_expert(l_export_distances_subset_table, pairs_file)
        cursor.close()
    # Get pairs
    # cbgs_restaurants_pairs = pd.read_sql_table('subset_cbg_restaurant_dist',
    #                                     engine)
    #result = engine.execute(delete_distances_subset_table_statement)
    connection = engine.connect()
    print(f'Getting devices and restaurants features.')
    # Get devices
    devices_table = pd.read_sql(devices_table_statement, engine)
    # Ensure cbg uniqueness 
    devices_table = devices_table.sort_values(['number_devices_residing'],
                                              ascending = False).groupby('cbg').head(1)
    
    # Count total choices made in the cbsa
    cbsa_total_choices = devices_table['number_devices_residing'].sum()
    cbsa_total_choices = 31 * cbsa_total_choices
    print(cbsa_total_choices)
    
    # Get restaurant characteristics
    restaurants_table = pd.read_sql(l_restaurants_table_statement, engine)
    restaurants_table['cbsa_total_choices'] = cbsa_total_choices
    
    # Get establishment counts by CBG:
    establishments_by_cbg = pd.read_sql(establishments_subset_by_cbg_table_statement,
                                        engine)
    
    print(f'Merging and adding to lists.')
    #cbgs_restaurants_pairs = pd.merge(cbgs_restaurants_pairs,
    #                                  devices_table,
    #                                  how = 'left', 
    #                                  on = 'cbg', 
    #                                  validate = 'many_to_one')
    
    restaurants_table = pd.merge(restaurants_table,
                                 devices_table, 
                                 how = 'left', 
                                 left_on = 'r_cbg', 
                                 right_on = 'cbg', 
                                 validate = 'many_to_one')
    
    restaurants_table.rename(columns = {'number_devices_residing': 'devices_nearby'}, 
                             inplace = True)
    restaurants_table.drop(columns = ['cbg', 'cbg_cbsa'], 
                           inplace = True)
    
    restaurants_table = pd.merge(restaurants_table,
                                 establishments_by_cbg,
                                 how = 'left',
                                 left_on = 'r_cbg',
                                 right_on = 'cbg',
                                 validate = 'many_to_one')
    restaurants_table.drop(columns = ['cbg'], 
                           inplace = True)
    restaurants_table['category1'] = restaurants_table['categories'].apply(lambda x: x[0]['alias'] if x else None)
    restaurants_table['n_categories'] = restaurants_table['categories'].apply(lambda x: len(x) if x else 0)
    restaurants_table.drop(columns = ['categories'], inplace = True)
    restaurants_table.fillna(value = {'est_nearby': 0}, inplace = True)
    
    # Export
    #cbgs_restaurants_pairs.to_csv(os.path.join(cbsa_folder_path, 
    #                                           f'pairs{selected_cbsa}.csv'),
    #                              index = False)
    restaurants_table.to_csv(os.path.join(cbsa_folder_path, 
                                          f'restaurants{selected_cbsa}.csv'),
                             index = False)
    
    engine.dispose()
    
    return None

################################################################################

In [ ]:
############################### Funcion run ####################################

for c in selected_cbsa_list:
    create_combinations(c)

################################################################################

In [ ]:
cbsa_list_file_path = os.path.join(output_folder_path, 'cbsa_list.csv')
with open(cbsa_list_file_path, 'a') as cbsa_list_file:
    for c in selected_cbsa_list:
        cbsa_list_file.write(f'{c}\n')
        
cbsa_list_1_file_path = os.path.join(output_folder_path, 'cbsa_list_6.csv')
with open(cbsa_list_1_file_path, 'a+') as cbsa_list_file:
    for c in selected_cbsa_list:
        cbsa_list_file.write(f'{c}\n')

In [ ]:
# Export list of cbsas of various sizes:

In [16]:
small_cbsa_list_file_path = os.path.join(output_folder_path, 'small_cbsa_list.csv')
small_cbsa_list = cbsa_table.loc[(cbsa_table['restaurant_count'] <= 2000) &
                                 (cbsa_table['restaurant_count'] >= 10)
                                ].reset_index(drop = True).loc[:,
                                                               'cbsa'
                                                              ].to_list()
with open(small_cbsa_list_file_path, 'w') as small_cbsa_list_file:
    for c in small_cbsa_list:
        small_cbsa_list_file.write(f'{c}\n')

In [17]:
medium_cbsa_list_file_path = os.path.join(output_folder_path, 'medium_cbsa_list.csv')
medium_cbsa_list = cbsa_table.loc[(cbsa_table['restaurant_count'] <= 4000) &
                                  (cbsa_table['restaurant_count'] > 2000)
                                 ].reset_index(drop = True).loc[:,
                                                                'cbsa'
                                                               ].to_list()
with open(medium_cbsa_list_file_path, 'w') as medium_cbsa_list_file:
    for c in medium_cbsa_list:
        medium_cbsa_list_file.write(f'{c}\n')

In [18]:
large_cbsa_list_file_path = os.path.join(output_folder_path, 'large_cbsa_list.csv')
large_cbsa_list = cbsa_table.loc[(cbsa_table['restaurant_count'] <= 6000) &
                                 (cbsa_table['restaurant_count'] > 4000)
                                ].reset_index(drop = True).loc[:,
                                                                'cbsa'
                                                               ].to_list()
with open(large_cbsa_list_file_path, 'w') as large_cbsa_list_file:
    for c in large_cbsa_list:
        large_cbsa_list_file.write(f'{c}\n')

In [19]:
extra_cbsa_list_file_path = os.path.join(output_folder_path, 'extra_cbsa_list.csv')
extra_cbsa_list = cbsa_table.loc[(cbsa_table['restaurant_count'] <= 20000) &
                                 (cbsa_table['restaurant_count'] > 6000)
                                ].reset_index(drop = True).loc[:,
                                                               'cbsa'
                                                              ].to_list()
with open(extra_cbsa_list_file_path, 'w') as extra_cbsa_list_file:
    for c in extra_cbsa_list:
        extra_cbsa_list_file.write(f'{c}\n')